## 1. 사용할 패키지 불러오기

In [2]:
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.nn import L1Loss
from data_gen.data_gen import dataset_generator
import torch
import torchvision.models as models
import torch.nn as nn
import numpy as np
from tqdm import tqdm


/home/jongwook95.lee/miniforge3/envs/vision_machine/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 데이터 불러오기

In [3]:
final_data = pd.read_excel('final_data.xlsx')
final_data.head()

,작가생존여부_사망,작가생존여부_생존,작가생존여부_알수없음,판매계절_가을,판매계절_겨울,판매계절_봄,판매계절_여름,재료_견본채색,재료_기타,재료_브론즈,...,판매처_케이옥션,판매처_헤럴드아트데이,가로,세로,작품 판매 횟수,판매가격,작가명,제목,이미지 고유 번호,이미지경로
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0.065372,0.065372,0.0,600000,임상진 Lim SangChin (1935~2013),무제,380410,data/image/11월_files/380410.jpg
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0.026429,0.026429,0.0,400000,정술원 Jung SulWon (1885~1959),화조,380460,data/image/11월_files/380460.jpg
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0.018878,0.018878,0.0,100000,정주상 Jeong JuSang (1925~2012),심정흥장 (선면),380491,data/image/11월_files/380491.jpg
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0.025351,0.025351,0.0,360000,이양원 Lee YangWon (1944~),풍속도,380417,data/image/11월_files/380417.jpg
4,0,1,0,1,0,0,0,0,1,0,...,0,0,0.019957,0.019957,0.0,240000,이외수 Lee OiSoo (1946~),사람과 사람들,380391,data/image/11월_files/380391.jpg


## 3. 이미지 별 RGB 평균 및 HSV 평균 변수 추가

In [20]:
final_data['R'] = 0
final_data['G'] = 0
final_data['B'] = 0
final_data['H'] = 0
final_data['S'] = 0
final_data['V'] = 0

for i in range(len(final_data['이미지경로'])):
    if i % 500 == 0:
        print(i)
    bgr = cv2.imread(final_data['이미지경로'][i])
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    

    final_data['R'][i] = np.mean(bgr[:, :, 2][bgr[:, :, 2] != 255])
    final_data['G'][i] = np.mean(bgr[:, :, 1][bgr[:, :, 1] != 255])
    final_data['B'][i] = np.mean(bgr[:, :, 0][bgr[:, :, 0] != 255])
    final_data['H'][i] = np.mean(hsv[:, :, 0][hsv[:, :, 0] != 255])
    final_data['S'][i] = np.mean(hsv[:, :, 1][hsv[:, :, 1] != 255])
    final_data['V'][i] = np.mean(hsv[:, :, 2][hsv[:, :, 2] != 255])
    

0


/tmp/ipykernel_12969/1360156975.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['R'][i] = np.mean(bgr[:, :, 2][bgr[:, :, 2] != 255])
/tmp/ipykernel_12969/1360156975.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['G'][i] = np.mean(bgr[:, :, 1][bgr[:, :, 1] != 255])
/tmp/ipykernel_12969/1360156975.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['B'][i] = np.mean(bgr[:, :, 0][bgr[:, :, 0] !

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000


In [21]:
final_data.to_excel('final_data_rgb_hsv.xlsx', index = False, encoding = 'euc-kr')